# The Ultimate Algo Trader (Neurobot 1.0)

## Introduction:
This jupyter notebook contains code for developing and testing a dynamic algorithmic trader based on Python and Machine learning strategy.

## Steps:
1. **Data preperation**: Access historical market data via Alpaca API and preprocess it for analysis.
2. **Strategy creation**: Code algorithmic trading strategy based on "TBD".
3. **Backtesting-Optimization**: Backtest the strategy using historical data and fine-tune parameters for better performance.
4. **Risk/Reward**: Calculate both the risk and the reward based on the entry price, position size, stop-loss and target price
5. **live Trading(Optional)**: Implement the strategy for paper trading on Alpaca.

**Tools and Libaries** 
- Python, Pandas, Numpy, "TBD"

#### Notes: 
- This notebook is for educational and experimental purposes only.

### Imports and Dependencies

In [ ]:
import os
import numpy as np
import random
import seaborn as sns
import pandas as pd
import yfinance as yf
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from scipy.interpolate import interp1d
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error


# Initialize python files and import functions
import stock_data as data
import algo_strategy as strategy
import nn_models as model
import backtesting as backtest

import warnings
warnings.filterwarnings('ignore')


In [ ]:
load_dotenv()

### Data Processing and collection

In [ ]:
# Fetch list of tickers
# Get select top picks based on monthly highest performing stocks in sp500 
#ticker_list = data.get_clusters_from_sp500(sp500_url = os.getenv("SP500_URL"))

# fetch_stock_data based on top picks from ticker_list
stock_data = data.fetch_stock_data('2018-01-12', '2024-03-14', tickers= ['NVDA'], timeframe='1Day')

In [ ]:
# data cleaning and organization
stock_df = stock_data #.rename(columns={'NVDA': 'NVDA close'})
stock_df['Daily Returns'] = stock_df['NVDA']['close'].pct_change()
stock_df = stock_df.dropna()
stock_df

#### Feauture Engineering - Time series Analysis

In [ ]:
stock_df['Cumulative Returns'] = (1 + stock_df['Daily Returns']).cumprod()

In [ ]:
stock_df['Daily Returns Lagged'] = stock_df['Daily Returns'].shift(-1)

In [ ]:
# set window sizes based on strategy
short_window = 5
long_window = 100

stock_df['SMA_Fast'] = stock_df['NVDA']['close'].rolling(window=short_window).mean()
stock_df['SMA_Slow'] = stock_df['NVDA']['close'].rolling(window=long_window).mean()
stock_df["EMA_Fast"] = stock_df['NVDA']["close"].ewm(span=short_window).mean()
stock_df["EMA_Fast"] = stock_df['NVDA']["close"].ewm(span=long_window).mean()
stock_df = stock_df.dropna()
stock_df

# later on -> trial different training windows using DateOffset()

### Algorithim Buy/Sell  Best signal selection

In [ ]:
# Initialize Signals 
signals = {
    "signal_one": strategy.strategy_one(stock_data),
    "signal_two": strategy.strategy_two(stock_data),
    "signal_three": strategy.strategy_three(stock_data),
    "signal_four": strategy.strategy_four(stock_data),
    "signal_five": strategy.strategy_five(stock_data) }

# Function for simple winning trading strategy 
def simple_winning(signals):
    best_strategy = max(signals, key=signals.get)
    winning_signal = signals[best_strategy]
    return winning_signal

# Function for strategy that combines signals using a majority vote
def majority_vote(signals):
    buy_signal = sum(1 for signal in signals.values() if signal == 1)    # "Buy"
    sell_signal = sum(1 for signal in signals.values() if signal == -1)  # "Sell"
    return 1 if buy_signal > sell_signal else -1 if sell_signal > buy_signal else 0  # "Buy", "Sell", or "Hold"

# Add signals to stock_df
stock_df['Majority_vote'] = majority_vote(signals)
stock_df['Simple_winning'] = simple_winning_(signals)

# Plot winning signal and hybrid


### Machine Learning Best Model Selection

In [ ]:
# Initialize Models
models = {
    "model_one": model.model_one(),
    "model_two": model.model_two(),
    "model_three": model.model_three(),
    "model_four": model.model_four(),
    "model_five": model.model_five() }

def select_best_model(models, X_train, y_train):
    # Define initial best score and best model
    best_model_name = None
    best_model = None
    best_accuracy = 0.0
    best_mse = float('inf')
    all_scores = {}
    all_mses = {}
    
    # Define the scoring metrics you want to use
    scoring = ['accuracy', 'precision', 'recall', 'f1_score']
    
    # Train and evaluate each model using cross-validation
    for name, model in models.items():
       
        metric_scores = {}
        
        # Evaluate model using cross-validation for each scoring metric
        for metric in scoring:
            scores = cross_val_score(model, X_train, y_train, cv=5, scoring=metric)
            
            metric_scores[metric] = scores
            
        # Calculate the mean scores
        mean_scores = {metric: scores.mean() for metric, scores in metric_scores.items()}
        mse_scores = -cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        mean_mse = mse_scores.mean()
        
        # Store the scores for the current model
        all_scores[name] = metric_scores
        all_mses[name] = mse_scores
        
        # Update best model if current model performs better based on accuracy or MSE
        if mean_mse < best_mse or (mean_mse == best_mse and mean_scores['accuracy'] > best_accuracy):
            best_accuracy = mean_scores['accuracy']
            best_mse = mean_mse
            best_model_name = name
            best_model = model
    
    return best_model_name, best_model, all_scores, all_mses

best_model_name, best_model, all_scores, all_mses = select_best_model(models, X_train, y_train)


### Backtesting - Best Model and Best Strategy

### Fundamental Analysis (Predictions and Plotting)

In [ ]:
 Import required libraries
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path

# Setting these options will allow for reviewing more of the DataFrames
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
NVDA = pd.read_csv(
    Path("../Resources/META.csv"),
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
META.head()
# Plot the DataFrame with hvplot["close"].hvplot()
META["Close"].hvplot
#COMPARE the SMA 50 plot line to an EMA50 plot line 
# Create a DataFrame with the index and Close column from the dataset
signals_df = META.loc[:, ["Close"]].copy()

# Set the short window and long windows
short_window = 50

# Create a short window SMA
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()

# Create a short window EMA
signals_df["EMA50"] = signals_df["Close"].ewm(span=short_window).mean()

# Review the DataFrame
signals_df.iloc[45:55, :]
#PLOT
# Visualize close price for the investment
security_close = signals_df[["Close"]].hvplot(
    line_color='Blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["SMA50", "EMA50"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
sma_ema_plot = security_close * moving_avgs
sma_ema_plot

#CREATE A TRADING ALGO!!!!!!
# Create a DataFrame with the index and Close column from the dataset
ema_signals_df = META.loc[:, ["Close"]].copy()

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
ema_signals_df["EMA50"] = signals_df["Close"].ewm(span=short_window).mean()
ema_signals_df["EMA100"] = signals_df["Close"].ewm(span=long_window).mean()

# Review the DataFrame
ema_signals_df.head()
# Set the Signal column
ema_signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the EMA50 is greater than (or crosses over) the EMA100
# where 0 is when the EMA50 is under the EMA100
ema_signals_df["Signal"][short_window:] = np.where(
    ema_signals_df["EMA50"][short_window:] > ema_signals_df["EMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
ema_signals_df["Entry/Exit"] = ema_signals_df["Signal"].diff()

# Review the DataFrame
ema_signals_df.head()
#PLOT!!!!!!
# Visualize entry position relative to close price
entry = ema_signals_df[ema_signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='Blue',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    xlabel= 'date',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = ema_signals_df[ema_signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color='Green',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = ema_signals_df[["Close"]].hvplot(
    line_color='Blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = ema_signals_df[["EMA50", "EMA100"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

#INCORPORATE THE FINTA PYTHON LIBRARY into the DMAC trading ALGORITHM 
# Import the finta library
from finta import TA
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
NVDA_df= pd.read_csv(
    Path("../Resources/META.csv"),
    
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
META_df.head()
#Generate the EMA Technical Indicator using FinTA
# Set the short window and long windows
short_window = 50
long_window = 100

# Add the EMA technical indicators for the short and long windows
META_df["Short"] = TA.EMA(NVDA_df, short_window)
META_df["Long"] = TA.EMA(NVDA_df, long_window)

# Review the DataFrame
META.iloc[45:105, :]
# Set the Signal column
META_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
META_df["Signal"][short_window:] = np.where(
    META_df["Short"][short_window:] > NVDA_df["Long"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
META_df["Entry/Exit"] = META_df["Signal"].diff()

# Review the DataFrame
META_df.iloc[95:105, :]
# Visualize entry position relative to close price
entry = META_df[META_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = NVDA_df[NVDA_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color="orange",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = META_df[["Close"]].hvplot(
    line_color="Yellow",
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = META_df[["Short", "Long"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot




 










In [ ]:
stock_df[['Daily Returns Lagged', 'Daily Returns']].corr()

# maybe not to use lagged strategy 
# consider DMAC? 

In [ ]:
stock_df['Cumulative Returns'].plot()

In [ ]:
# Print the selected best model name
print("Selected Best Model:", best_model_name)

# Print the cross-validation scores for each model and each metric
for name, scores in all_scores.items():
    print("Model:", name)
    for metric, metric_scores in scores.items():
        print(f"Mean {metric.capitalize()} Score:", metric_scores.mean())

# Print the cross-validation MSE scores for each model
for name, mse_scores in all_mses.items():
    print("Model:", name)
    print("Cross-Validation MSE Scores:", mse_scores)

### Risk management and Rewards

### Logic to place trade (Optional)

In [ ]:
# From Algo Trading 3 live 

# Submit order
api.submit_order(
    symbol="META", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="gtc", 
    type="limit", 
    limit_price=limit_amount
